In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

In [2]:
MODEL_PATH = '../models/original_model.tflite'
CLASS_NAMES = sorted(os.listdir('../dataset/products/validation')) # directory names are the class names

In [3]:
def load_and_preprocess_image(filename):
    image = tf.keras.utils.load_img(filename, target_size=(224, 224, 3))
    image = tf.keras.utils.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image
    

In [4]:
interpreter = tf.lite.Interpreter(MODEL_PATH)
interpreter.allocate_tensors()

Single image

In [5]:
TEST_IMAGE_PATH = 'test.jpg'
image = load_and_preprocess_image(TEST_IMAGE_PATH)

In [6]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [7]:
interpreter.set_tensor(input_details[0]['index'], image)

In [8]:
%timeit interpreter.invoke()

25.8 ms ± 70 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
predictions = interpreter.get_tensor(output_details[0]['index'])
predictions

array([[2.6806073e-11, 1.1282087e-12, 5.1287110e-08, 1.1686058e-06,
        6.3463885e-16, 6.8896993e-13, 1.2214249e-06, 5.1749035e-12,
        4.5067642e-13, 3.9826781e-17, 2.5354030e-11, 3.8585881e-11,
        3.9000976e-11, 1.5328704e-12, 9.9999750e-01]], dtype=float32)

In [10]:
print(CLASS_NAMES[np.argmax(predictions)], " prediction: " ,np.max(predictions))

Tomato  prediction:  0.9999975


Many images

In [11]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

Prepare generator, that returns tuple (images, classes). Image shape is (batch_size x pixels x pixels x three_colors). Classes shape is (batch_size x classes_number)

In [12]:
test_generator = test_datagen.flow_from_directory(
    '../dataset/products/test',
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical'
)

Found 3000 images belonging to 15 classes.


In [13]:
val_matrix = np.zeros((15, 15))
total_time = 0
TEST_LIMIT = 1000

for iteration, (image, class_) in enumerate(test_generator):
    class_idx = np.argmax(class_)
    interpreter.set_tensor(input_details[0]['index'], image)
    
    start_time = time.time()
    interpreter.invoke()
    total_time += time.time() - start_time
    predictions = interpreter.get_tensor(output_details[0]['index'])
    val_matrix[class_idx] += predictions.flatten()
    
    if (iteration > 0 and iteration % 100 == 0):
        print("iteration: ", iteration)
        print("avg time: ", 1000 * total_time / (iteration + 1), "ms")
        print("accuracy: ", np.sum(np.diag(val_matrix)/np.sum(val_matrix)))
        print()

    if iteration >= TEST_LIMIT:
        break

print("Total time:", total_time, "s")

iteration:  100
avg time:  25.68945082107393 ms
accuracy:  0.9421994559640713

iteration:  200
avg time:  25.719411337553566 ms
accuracy:  0.9594122923686929

iteration:  300
avg time:  25.7530584683846 ms
accuracy:  0.9593759604631747

iteration:  400
avg time:  25.788022990238638 ms
accuracy:  0.9603865603603265

iteration:  500
avg time:  25.87743481238207 ms
accuracy:  0.959666139515139

iteration:  600
avg time:  25.995644475775034 ms
accuracy:  0.9631133746887699

iteration:  700
avg time:  26.00040177305823 ms
accuracy:  0.9619821793583851

iteration:  800
avg time:  25.99955557586251 ms
accuracy:  0.9635474601463829

iteration:  900
avg time:  26.01253417435285 ms
accuracy:  0.9626253277719484

iteration:  1000
avg time:  25.984514247882853 ms
accuracy:  0.9597899847975085

Total time: 26.010498762130737 s
